In [ ]:
pair_prompt = f'''You are a Policy Assistant Evaluator on behalf of the User who set a Policy Assistant Goal. Your task is to evaluate the Current Policy Assistant Responses by selecting 1 response that fulfills MORE of the Policy Assistant Success Criteria provided better than the unselected response, or returning -1 if both responses are equally successful and it is impossible to differentiate which response is better. Show your step-by-step reasoning inside <think> and </think> tags, then provide your final answer inside <answer> and </answer> tags as a response index (0, 1, or -1) for which response fulfills more of the criteria or if they are both equally successful. Note: fulfilling more higher-priority criteria should be considered as fulfilling more of the success criteria than fulfilling more lower-priority criteria.

                    
# Policy Assistant Goal: 
Get really good at math. 


# Policy Assistant Success Criteria:
The Policy Assistant Response should be (in order of priority):

1) maximizing user and Policy Assistant benefit and helpfulness and minimizing user and Policy Assistant risk, harm, and unwanted outcomes.

2) logically correct - the final answer to the problem must be as correct as possible, and as de-risked of being the wrong final answer as much as possible

3) clear - the final answer should be unambiguous and directly state the correct result

4) concise - the final answer should be free of unnecessary details while still conveying all essential information

5) efficient reasoning - even if the reasoning wanders, the path to the final answer should be streamlined and focused

6) error minimization - the response should be robust, minimizing the chance of errors in the final answer

7) insightfulness - the response should reflect a deep understanding of the problem and apply appropriate techniques

8) formatted - the final answer should be between <answer> tags

                    
# POLICY ASSISTANT PROMPT: 
"{prompt}"


# CURRENT POLICY ASSISTANT RESPONSES:
Policy Assistant Response 0: 
"{resp1}"


Policy Assistant Response 1: 
"{resp2}"'''

data_pair = {
    "temperature": 0.0,
    "max_tokens": 4096,
    "messages": [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": pair_prompt,
        }]
    }],
    "stop": ["<|im_end|>"],
    "n": 1,
    "model": "dev-gpt-4o-vision-2024-05-13"
}
pair_data_list.append(data_pair)
pair_indices_list.append((idx1, idx2))
pair_log_entries.append({
    "prompt": prompt,
    "pair_indices": (idx1, idx2),
    "pair_prompt": pair_prompt,
    "evaluator_response": None,
    "expected_answer": extras[idx].get("answer", "N/A"),
    "response_0_match_expected": outputs[idx1]["iscorrect"],
    "response_1_match_expected": outputs[idx2]["iscorrect"],
})
# For an odd leftover, assign default reward 0.5.
if len(indices) % 2 == 1:
leftover = indices[-1]
pairwise_rewards[leftover] = 0.5
pair_log_entries.append({
    "prompt": prompt,
    "pair_indices": (leftover,),
    "pair_prompt": "N/A (odd leftover)",
    "evaluator_response": "N/A",
    "decision": "default 0.5",
    "expected_answer": extras[idx].get("answer", "N/A"),
    "response_0_match_expected": outputs[leftover]["iscorrect"],
})

# Define a remote function for GPT-4o pair evaluation.
@ray.remote(num_cpus=1)
def evaluate_policy_pair(data):
url = "https://skilled-chigger-mentally.ngrok-free.app/api/send_request"
max_retries = 3
for attempt in range(max_retries):
try:
response = requests.post(url, json=data)
return response.json()['choices'][0]['message']['content']
except Exception as e:
logger.error(f"Error in evaluate_policy_pair attempt {attempt+1}: {e}")
if attempt < max_retries - 1:
    import time
    time.sleep(2)
return ""

In [ ]:
data_pair = {
    "temperature": 0.0,
    "max_tokens": 4096,
    "messages": [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": pair_prompt,
        }]
    }],
    "stop": ["<|im_end|>"],
    "n": 1,
    "model": "dev-gpt-4o-vision-2024-05-13"
}